In [ ]:
from IPython.display import display, Math
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import scipy as scipy    

# Minimal Example of a Buckling Rod with Axial-transversal Coupling
Supplementary material to the [GAMMAS](https://www.bibliothek.tu-chemnitz.de/ojs/index.php/gammas) article **From Problem to Failure -- Insights from the Eigenvalue Problem of the Stiffness Matrix in Non-linear Structural Analysis** by
[Chiara Hergl](https://orcid.org/0000-0002-4016-9113),
[Dominik Kern](https://orcid.org/0000-0002-1958-2982),
[Thomas Nagel](https://orcid.org/0000-0001-8459-4616)

![Euler buckling case 2](euler_buckling_horizontal.svg)

Non-dimensional potential energy for coupled beam bending (transversal displacement $w$, axial displacement $u$) with axial force $F$ and stabilizing spring (stiffness $k$)    
$\Pi = \frac{1}{2}\int_0^1 w''(x)^2 + G \left(u' + \frac{1}{2} w'^2\right)^2 \mathrm{d}x +\frac{1}{2}kw(X)^2 + F u(1)$,

and following relations to the dimensional variables 
- $G = \dfrac{A l^2}{I}$ squared slenderness ratio,
- $k = \dfrac{k_\mathrm{dim}l}{F_\mathrm{ref}}$,
- $F = \dfrac{F_\mathrm{dim}}{F_\mathrm{ref}}$
with $F_\mathrm{ref} = \dfrac{EI}{l^2}$, note that first buckling load without spring ($k=0$) is $F_\mathrm{crit} = \pi^2 F_\mathrm{ref}$.

Ritz' method with discretization $w(x)\approx \sum\limits_{j=1}^{J} \sin\bigl(j\pi x \bigr)$ and $u\approx \hat{u}x$, here $J=2$.

## Symbolic calculations

Apply Ritz discretization (3 DoF: 1 axial and 2 transversal) and evaluate first variation of the potential energy

In [ ]:
x, X, k, j, F, W1, W2, dW1, dW2, U1, dU1, G  = sp.symbols('x X k j F W1 W2 dW1 dW2 U1 dU1 G')

In [ ]:
w = sp.sin(j*sp.pi*x) 

W = W1*w.subs(j,1) + W2*w.subs(j,2)
Wx = W.diff(x)
Wxx = Wx.diff(x)

dW = dW1*w.subs(j,1) + dW2*w.subs(j,2)
dWx = dW.diff(x)
dWxx = dWx.diff(x)

U = U1*x
Ux = U.diff(x)

dU = dU1*x
dUx = dU.diff(x)

dPi = sp.integrate(Wxx*dWxx + G*(Ux+sp.Rational(1,2)*Wx*Wx)*(dUx+Wx*dWx), (x,0,1) ) + k*W.subs(x,X)*dW.subs(x,X) + F*U.subs(x,1)
#display(dPi)

Generalized forces, conjugated to $w_1$, $w_2$ and $u_1$, respectively.

In [ ]:
Q1 = sp.simplify(dPi.diff(W1))
display(Math(f'Q_1 = {sp.latex(Q1)}'))
Q2 = sp.simplify(dPi.diff(W2))
display(Math(f'Q_2 = {sp.latex(Q2)}'))
Q3 = sp.simplify(dPi.diff(U1))
display(Math(f'Q_3 = {sp.latex(Q3)}'))

Stiffness matrix, remember $\sin(2x)=2\sin(x)\cos(x)$ to recognize its symmetry

In [ ]:
K11 = sp.expand(Q1).coeff(dW1, 1)
K12 = sp.expand(Q1).coeff(dW2, 1)
K13 = sp.expand(Q1).coeff(dU1, 1)

K21 = sp.expand(Q2).coeff(dW1, 1)
K22 = sp.expand(Q2).coeff(dW2, 1)
K23 = sp.expand(Q2).coeff(dU1, 1)

K31 = sp.expand(Q3).coeff(dW1, 1)
K32 = sp.expand(Q3).coeff(dW2, 1)
K33 = sp.expand(Q3).coeff(dU1, 1)

K = sp.Matrix(((K11, K12, K13), (K21, K22, K23), (K31, K32, K33)))
display(Math(f'K = {sp.latex(K)}'))

Stiffness matrix before buckling, i.e. without bending deformation $w(x)=0$.

In [ ]:
nDoF = 3
Wzero = 0
Kzero = K.subs([(W1, Wzero), (W2, Wzero)])
display(Math(f'C_0 = {sp.latex(Kzero)}'))

Buckling occurs when the determinant of the stiffness matrix vanishes.

In [ ]:
char_eq = sp.simplify(sp.Determinant(K).doit())
buckling_state = sp.solve(char_eq, U1)
print(str(len(buckling_state)) + " buckling states (as many as transversal DoF)")
U_1 = sp.simplify(buckling_state[0])
U_2 = sp.simplify(buckling_state[1])
Uzero_1 = U_1.subs([(W1, Wzero), (W2, Wzero)])
Uzero_2 = U_2.subs([(W1, Wzero), (W2, Wzero)])

## Numerical evaluations

common value for the slenderness ratio and a wide range of spring stiffnesses

In [ ]:
N = 101
k_grid = np.linspace(0, 300, N)   # spring stiffness
Gnum = 100**2   # squarred slenderness ratio
num_zero = 1e-10

F_lower_sym = np.zeros((N,1))
F_higher_sym = np.zeros((N,1))

F_lower_asym = np.zeros((N,1))
F_higher_asym = np.zeros((N,1))
gev_lower = np.zeros((N,3))
gev_higher = np.zeros((N,3))
sev_lower = np.zeros((N,3))
sev_higher = np.zeros((N,3))

### stabilizing spring in centre position ($X=\frac{1}{2}$) _symmetric case_

In [ ]:
Xsym = 0.5

Set up generalized eigenvalue problem $\mathbf{A}\mathbf{w} = x\mathbf{B}\mathbf{w}$ from nonlinear EVP and compare with linear EVP at $U=U_\mathrm{crit}$


In [ ]:
for n,knum in enumerate(k_grid):
    Kzero_sym = Kzero.subs([(X,Xsym),(k,knum),(G,Gnum)])
    K33num = Kzero_sym.row(2).col(2)[0]  # strictly speaking, here we'd need matrix-vector multiplication, but we know matrix structure
    
    # C = A + B*U1, since we specifically know the matrix structure for this example problem   
    Azero_sym = sp.Matrix([[( Kzero_sym[i,j].coeff(U1,0).evalf()) for i in range(nDoF)] for j in range(nDoF)])
    Bzero_sym = sp.Matrix([[(-Kzero_sym[i,j].coeff(U1,1).evalf()) for i in range(nDoF)] for j in range(nDoF)])
    Azero_num = np.array(Azero_sym).astype(np.float64)
    Bzero_num = np.array(Bzero_sym).astype(np.float64)
    eigvals, eigvecs = scipy.linalg.eig(Azero_num, b=Bzero_num)
    
    # sort by eigenvalues
    if np.abs(eigvals[2]) < np.abs(eigvals[0]) or np.abs(eigvals[2]) < np.abs(eigvals[1]):
        print("Warning, for k={} assumption of third eigenvalue being infinite is not fulfilled!".format(k))
    
    if np.abs(eigvals[0]) < np.abs(eigvals[1]):
        F_lower_sym[n]  = eigvals[0].real*K33num.evalf()
        F_higher_sym[n] = eigvals[1].real*K33num.evalf()
    else:
        F_lower_sym[n]  = eigvals[1].real*K33num.evalf()
        F_higher_sym[n] = eigvals[0].real*K33num.evalf()

Plot buckling load for a range of spring stiffnesses

In [ ]:
plt.plot(k_grid,np.abs(F_lower_sym),'b', k_grid,np.abs(F_higher_sym),'r')
plt.xlabel("stiffness");
plt.ylabel("buckling load");
plt.legend(["|F_1|", "|F_2|"]);

Note that the horizontal line, independent on color, corresponds to the second mode shape and the sloped line to the first mode shape. The physics behind this plot are that, the spring only stabilizes against first mode buckling and is ineffective for the second mode (node point). There is one value of the spring stiffness, when both buckling loads are equal.

### stabilizing spring in off-centre position ($X\ne\frac{1}{2}$) _asymmetric case_
By _asymmetry_ we refer to the axial direction, as in transversal direction symmetry is preserved, i.e. no difference in bending to the right or to the left.

In [ ]:
Xasym = 0.55   

In [ ]:
for n,knum in enumerate(k_grid):
    Kzero_asym = Kzero.subs([(X,Xasym),(k,knum),(G,Gnum)])
    K33num = Kzero_asym.row(2).col(2)[0]
    
    Azero_asym = sp.Matrix([[( Kzero_asym[i,j].coeff(U1,0).evalf()) for i in range(nDoF)] for j in range(nDoF)])
    Bzero_asym = sp.Matrix([[(-Kzero_asym[i,j].coeff(U1,1).evalf()) for i in range(nDoF)] for j in range(nDoF)])
    Azero_anum = np.array(Azero_asym).astype(np.float64)
    Bzero_anum = np.array(Bzero_asym).astype(np.float64)
    eigvals, eigvecs = scipy.linalg.eig(Azero_anum, b=Bzero_anum)   # generalized Eigenvalue problem  
    
    # sort by eigenvalues
    if np.abs(eigvals[2]) < np.abs(eigvals[0]) or np.abs(eigvals[2]) < np.abs(eigvals[1]):
        print("Warning, for k={} assumption of third eigenvalue being infinite is not fulfilled!".format(k))
        
    if np.abs(eigvals[0]) < np.abs(eigvals[1]): 
        U1lower  = eigvals[0]
        F_lower_asym[n] = eigvals[0].real*K33num.evalf()
        gev_lower[n,:] = eigvecs[:,0]
        U1higher = eigvals[1]
        F_higher_asym[n] = eigvals[1].real*K33num.evalf()
        gev_higher[n,:] = eigvecs[:,1]
    
    else:
        U1lower  = eigvals[1]
        F_lower_asym[n] = eigvals[1].real*K33num.evalf()
        gev_lower[n,:] = eigvecs[:,1]
        U1higher = eigvals[0]
        F_higher_asym[n] = eigvals[0].real*K33num.evalf()
        gev_higher[n,:] = eigvecs[:,0]

    # compute eigenvector (zero eigenvalue) at lower buckling load
    Kzero_lower =  np.matrix(Kzero_asym.subs(U1, U1lower).evalf()).astype(np.float64)
    standard_eigvals, standard_eigvecs = scipy.linalg.eigh(Kzero_lower)
    # pick eigenvector of eigenvalue = 0
    i_min = np.argmin(np.abs(standard_eigvals))
    if np.abs(standard_eigvals[i_min]) > num_zero:
        print("Warning, minimal eigenvalue (linear EVP) at buckling must be zero!")
    else:
        sev_lower[n,:] = standard_eigvecs[:,i_min]

    # compute eigenvector (zero eigenvalue) at higher buckling load
    Kzero_higher =  np.matrix(Kzero_asym.subs(U1, U1higher).evalf()).astype(np.float64)
    standard_eigvals, standard_eigvecs = scipy.linalg.eigh(Kzero_higher)
    # pick eigenvector of eigenvalue = 0
    i_min = np.argmin(np.abs(standard_eigvals))
    if np.abs(standard_eigvals[i_min]) > num_zero:
        print("Warning, minimal eigenvalue (linear EVP) at buckling must be zero!")
    else:
        sev_higher[n,:] = standard_eigvecs[:,i_min]

In [ ]:
plt.plot(k_grid,np.abs(F_lower_asym),'b', k_grid,np.abs(F_higher_asym),'r')
plt.xlabel("stiffness");
plt.ylabel("buckling load");
plt.legend(["|F_1|", "|F_2|"]);

There is no common value of two buckling modes for any spring stiffness.

#### Eigenvectors

##### Nonlinear EVP lower mode

In [ ]:
plt.plot(k_grid, np.abs(gev_lower[:,0]),'b', k_grid, np.abs(gev_lower[:,1]),'b--', k_grid, np.abs(gev_lower[:,2]),'b:');
plt.xlabel('$k$');
plt.legend(['$|w_1|$', '$|w_2|$', '$|u_1|$']);

##### Linear EVP lower mode

In [ ]:
plt.plot(k_grid, np.abs(sev_lower[:,0]),'c', k_grid, np.abs(sev_lower[:,1]),'c--', k_grid, np.abs(sev_lower[:,2]),'c:');
plt.xlabel('$k$');
plt.legend(['$|w_1|$', '$|w_2|$', '$|u_1|$']);

For the lower buckling load there is a transition from the first mode shape to the second. Both eigenvectors, from the nonlinear and from the linear EVP, are identical as expected. 

##### Nonlinear EVP higher mode

In [ ]:
plt.plot(k_grid, np.abs(gev_higher[:,0]),'r', k_grid, np.abs(gev_higher[:,1]),'r--', k_grid, np.abs(gev_higher[:,2]),'r:');
plt.xlabel('$k$');
plt.legend(['$|w_1|$', '$|w_2|$', '$|u_1|$']);

##### Linear EVP higher mode

In [ ]:
plt.plot(k_grid, np.abs(sev_higher[:,0]),'m', k_grid, np.abs(sev_higher[:,1]),'m--', k_grid, np.abs(sev_higher[:,2]),'m:');
plt.xlabel('$k$');
plt.legend(['$|w_1|$', '$|w_2|$', '$|u_1|$']);

For the higher buckling load it is the opposite transition than for the lower buckling load. Again, both eigenvectors match.

#### Plots of the buckling shapes for different values of the spring stiffness

In [ ]:
x_grid = np.linspace(0, 1) 
index_k = [1, 50, 100]    # low, intermediate and high spring stiffness (manual selection)
for i_k in index_k:
    w_buckling_lower_load  = lambda x: gev_lower[i_k, 0] * np.sin(np.pi*x) + gev_lower[i_k, 1] * np.sin(2*np.pi*x) 
    w_buckling_higher_load = lambda x: gev_higher[i_k, 0] * np.sin(np.pi*x) + gev_higher[i_k, 1] * np.sin(2*np.pi*x) 
    plt.plot(x_grid, w_buckling_lower_load(x_grid),'b')
    plt.plot(x_grid, w_buckling_higher_load(x_grid),'r')
    plt.xlabel('x')
    plt.ylabel('w')
    plt.ylim([-1.25,1.25])
    plt.legend(['F_1 (lower)', 'F_2 (higher)'])
    plt.title('k={:.2f}'.format(k_grid[i_k]))
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1, Xasym], ['0', '0.2', '0.4', '0.6', '0.8', '1', 'X'])
    plt.show(); 